In [1]:
import os
import sys
import transbigdata as tbd
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plot_map
import re
import numpy as np
from pathlib import Path
import time
import numpy as np

In [10]:
DBS_all = pd.read_csv(r'D:/DBS_data/union/dbs_csv/DBS_all.csv', header = None)

In [12]:
DBS_all.columns =['start_time', 'start_lat_bd', 'end_time', 'end_lng_bd', 'start_lng_bd','end_lat_bd','dist','time','velocity','minute','hour','day','start_lng','start_lat','end_lng','end_lat']

In [18]:
DBS_all.to_csv('DBS_all.csv')

In [42]:
DBS_all['counter'] = range(len(DBS_all))

In [100]:
#读取小区AOI
home = gpd.read_file(r'D:\house_work\json\xiaoqu.json')
#读取写字楼AOI
workplace = gpd.read_file(r'D:\house_work\json\xiezilou.json')
#读取地铁BnR
BnR = gpd.read_file(r'D:\house_work\json\metro.json')

In [102]:
DBS_all

,start_time,start_lat_bd,end_time,end_lng_bd,start_lng_bd,end_lat_bd,dist,time,velocity,minute,hour,day,start_lng,start_lat,end_lng,end_lat,counter
0,2021/2/1 6:01,22.649138,2021/2/1 6:03,114.025076,114.026905,22.650569,246.062284,120,7.381869,1,6,1,114.015404,22.645586,114.013579,22.647022,0
1,2021/2/1 6:01,22.620631,2021/2/1 6:03,114.060683,114.060174,22.617641,336.627187,120,10.098816,1,6,1,114.048609,22.617007,114.049118,22.614016,1
2,2021/2/1 6:01,22.530571,2021/2/1 6:03,114.025896,114.024123,22.529488,218.251016,120,6.547530,1,6,1,114.012634,22.527041,114.014402,22.525953,2
3,2021/2/1 6:01,22.530489,2021/2/1 6:03,114.025813,114.024041,22.529406,218.251107,120,6.547533,1,6,1,114.012552,22.526959,114.014320,22.525871,3
4,2021/2/1 6:01,22.536203,2021/2/1 6:03,114.109418,114.107686,22.535712,186.117173,120,5.583515,1,6,1,114.096110,22.532556,114.097844,22.532066,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7281175,2021/2/7 19:59,22.505018,2021/2/7 20:09,114.050264,114.060994,22.519296,1932.850000,600,11.597100,59,19,7,114.049434,22.501402,114.040000,22.515701,7281175
7281176,2021/2/7 18:17,22.601245,2021/2/7 18:19,114.035004,114.036966,22.604470,411.390000,120,12.341700,17,18,7,114.025443,22.597674,114.020000,22.600904,7281176
7281177,2021/2/7 20:04,22.678594,2021/2/7 20:22,113.958917,113.949898,22.683814,1092.300000,1080,3.641000,4,20,7,113.938564,22.675250,113.950000,22.680449,7281177
7281178,2021/2/7 22:27,22.573340,2021/2/7 22:54,113.949983,113.942744,22.568465,919.980000,1620,2.044400,27,22,7,113.931420,22.570026,113.940000,22.565138,7281178


In [103]:
# 识别起点
# 识别出发地和目的地
# 家到地铁站，时间6-9点
start_6_9 = DBS_all[DBS_all['hour'].map(lambda x:6<=x<=9)]

In [116]:
start_6_9

,start_time,start_lat_bd,end_time,end_lng_bd,start_lng_bd,end_lat_bd,dist,time,velocity,minute,hour,day,start_lng,start_lat,end_lng,end_lat,counter
0,2021/2/1 6:01,22.649138,2021/2/1 6:03,114.025076,114.026905,22.650569,246.062284,120,7.381869,1,6,1,114.015404,22.645586,114.013579,22.647022,0
1,2021/2/1 6:01,22.620631,2021/2/1 6:03,114.060683,114.060174,22.617641,336.627187,120,10.098816,1,6,1,114.048609,22.617007,114.049118,22.614016,1
2,2021/2/1 6:01,22.530571,2021/2/1 6:03,114.025896,114.024123,22.529488,218.251016,120,6.547530,1,6,1,114.012634,22.527041,114.014402,22.525953,2
3,2021/2/1 6:01,22.530489,2021/2/1 6:03,114.025813,114.024041,22.529406,218.251107,120,6.547533,1,6,1,114.012552,22.526959,114.014320,22.525871,3
4,2021/2/1 6:01,22.536203,2021/2/1 6:03,114.109418,114.107686,22.535712,186.117173,120,5.583515,1,6,1,114.096110,22.532556,114.097844,22.532066,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7228666,2021/2/7 9:13,22.583089,2021/2/7 9:23,113.881733,113.878764,22.572227,1245.670000,600,7.474020,13,9,7,113.867400,22.579803,113.870000,22.568944,7228666
7230303,2021/2/7 8:36,22.539567,2021/2/7 8:40,113.969598,113.968475,22.538344,178.310000,240,2.674650,36,8,7,113.957117,22.536199,113.960000,22.534973,7230303
7274104,2021/2/7 9:09,22.617248,2021/2/7 9:14,114.027781,114.023234,22.612805,679.680000,300,8.156160,9,9,7,114.011742,22.613712,114.020000,22.609257,7274104
7280107,2021/2/7 8:32,22.660614,2021/2/7 8:42,114.069196,114.074331,22.661064,529.280000,600,3.175680,32,8,7,114.062749,22.656962,114.060000,22.657418,7280107


In [115]:
start_6_9.to_csv('start_6_9.csv')

In [118]:
# 从6-9点的出行中筛选起点是在家的
HtoM_select = tbd.clean_outofshape(start_6_9, home, col=['start_lng', 'start_lat'], accuracy=5) #起点在家
# 从6-9点,起点在家的出行中筛选终点是地铁站的
HtoM = tbd.clean_outofshape(HtoM_select, BnR, col=['end_lng', 'end_lat'], accuracy=5) #终点在地铁站

In [119]:
HtoM_select

,start_time,start_lat_bd,end_time,end_lng_bd,start_lng_bd,end_lat_bd,dist,time,velocity,minute,hour,day,start_lng,start_lat,end_lng,end_lat,counter
0,2021/2/1 6:02,22.549655,2021/2/1 6:04,114.124020,114.128401,22.549093,454.171495,120,13.625145,2,6,1,114.116850,22.546026,114.112463,22.545459,8
1,2021/2/1 7:59,22.549614,2021/2/1 8:06,114.120717,114.128440,22.547558,825.356175,420,7.074482,59,7,1,114.116889,22.545986,114.109155,22.543921,66663
2,2021/2/2 8:30,22.549632,2021/2/2 8:42,114.116326,114.128442,22.539250,1697.260000,720,8.486300,30,8,2,114.116891,22.546003,114.104759,22.535609,2010798
3,2021/2/5 6:53,22.549634,2021/2/5 7:02,114.115772,114.128422,22.538512,1793.630000,540,11.957533,53,6,5,114.116871,22.546006,114.104204,22.534871,4815156
4,2021/2/5 6:53,22.549620,2021/2/5 7:02,114.115758,114.128408,22.538499,1793.630000,540,11.957533,53,6,5,114.116857,22.545992,114.104190,22.534857,5250916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392930,2021/2/7 9:44,22.501621,2021/2/7 10:27,113.923434,113.937935,22.502693,1494.390000,2580,2.085195,44,9,7,113.926619,22.498319,113.910000,22.499409,7192901
392931,2021/2/7 9:57,22.495665,2021/2/7 10:37,113.934768,113.924679,22.481745,1862.840000,2400,2.794260,57,9,7,113.913368,22.492380,113.920000,22.478447,7193673
392932,2021/2/7 8:31,22.600644,2021/2/7 8:34,114.032439,114.038130,22.608446,1045.900000,180,20.918000,31,8,7,114.026606,22.597070,114.020000,22.604886,7196313
392933,2021/2/7 8:52,22.620062,2021/2/7 9:07,114.061116,114.064227,22.625967,730.180000,900,2.920720,52,8,7,114.052657,22.616431,114.050000,22.622340,7204076


In [120]:
HtoM

,start_time,start_lat_bd,end_time,end_lng_bd,start_lng_bd,end_lat_bd,dist,time,velocity,minute,hour,day,start_lng,start_lat,end_lng,end_lat,counter
0,2021/2/1 6:02,22.549655,2021/2/1 6:04,114.124020,114.128401,22.549093,454.171495,120,13.625145,2,6,1,114.116850,22.546026,114.112463,22.545459,8
1,2021/2/5 9:20,22.547734,2021/2/5 9:29,114.124045,114.133885,22.549115,1022.040000,540,6.813600,20,9,5,114.122344,22.544114,114.112488,22.545481,5444222
2,2021/2/1 6:01,22.582325,2021/2/1 6:04,113.870969,113.875366,22.579923,524.561056,180,10.491221,1,6,1,113.863995,22.579036,113.859588,22.576631,32
3,2021/2/1 6:01,22.551113,2021/2/1 6:03,114.142984,114.142228,22.552459,168.579088,120,5.057373,1,6,1,114.130704,22.547507,114.131461,22.548854,35
4,2021/2/1 6:02,22.560429,2021/2/1 6:06,114.106867,114.101345,22.562984,634.235212,240,9.513528,2,6,1,114.089764,22.556779,114.095290,22.559335,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89757,2021/2/7 9:56,22.523861,2021/2/7 10:02,113.925777,113.925193,22.515532,928.130000,360,9.281300,56,9,7,113.913880,22.520575,113.910000,22.512245,7190883
89758,2021/2/7 9:39,22.587579,2021/2/7 10:05,113.857953,113.874467,22.585484,1711.320000,1560,3.949200,39,9,7,113.863094,22.584289,113.850000,22.582177,7190898
89759,2021/2/7 9:23,22.582637,2021/2/7 9:26,113.960705,113.964378,22.581505,397.530000,180,7.950600,23,9,7,113.953025,22.579276,113.950000,22.578153,7191035
89760,2021/2/7 9:47,22.538830,2021/2/7 10:09,114.090853,114.091832,22.545637,763.560000,1320,2.082436,47,9,7,114.080250,22.535181,114.080000,22.541988,7191563


In [122]:
# 剔除HtoM
#上下合并表格，并重置索引
start_6_9_HM = pd.concat([start_6_9,HtoM], ignore_index=True, verify_integrity=True,sort=True) 
#删除合并后姓名里有重复的所有行
start_6_9_HM.drop_duplicates(subset=['counter'],keep=False,inplace=True) #subset=['counter']表示检查counter这一列是否有重复内容，keep=False表示不保留重复内容

In [123]:
start_6_9_HM

,counter,day,dist,end_lat,end_lat_bd,end_lng,end_lng_bd,end_time,hour,minute,start_lat,start_lat_bd,start_lng,start_lng_bd,start_time,time,velocity
0,0,1,246.062284,22.647022,22.650569,114.013579,114.025076,2021/2/1 6:03,6,1,22.645586,22.649138,114.015404,114.026905,2021/2/1 6:01,120,7.381869
1,1,1,336.627187,22.614016,22.617641,114.049118,114.060683,2021/2/1 6:03,6,1,22.617007,22.620631,114.048609,114.060174,2021/2/1 6:01,120,10.098816
2,2,1,218.251016,22.525953,22.529488,114.014402,114.025896,2021/2/1 6:03,6,1,22.527041,22.530571,114.012634,114.024123,2021/2/1 6:01,120,6.547530
3,3,1,218.251107,22.525871,22.529406,114.014320,114.025813,2021/2/1 6:03,6,1,22.526959,22.530489,114.012552,114.024041,2021/2/1 6:01,120,6.547533
4,4,1,186.117173,22.532066,22.535712,114.097844,114.109418,2021/2/1 6:03,6,1,22.532556,22.536203,114.096110,114.107686,2021/2/1 6:01,120,5.583515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260146,7228666,7,1245.670000,22.568944,22.572227,113.870000,113.881733,2021/2/7 9:23,9,13,22.579803,22.583089,113.867400,113.878764,2021/2/7 9:13,600,7.474020
2260147,7230303,7,178.310000,22.534973,22.538344,113.960000,113.969598,2021/2/7 8:40,8,36,22.536199,22.539567,113.957117,113.968475,2021/2/7 8:36,240,2.674650
2260148,7274104,7,679.680000,22.609257,22.612805,114.020000,114.027781,2021/2/7 9:14,9,9,22.613712,22.617248,114.011742,114.023234,2021/2/7 9:09,300,8.156160
2260149,7280107,7,529.280000,22.657418,22.661064,114.060000,114.069196,2021/2/7 8:42,8,32,22.656962,22.660614,114.062749,114.074331,2021/2/7 8:32,600,3.175680


In [124]:
# 从6-9点的出行中筛选起点是在地铁站的
MtoW = tbd.clean_outofshape(start_6_9_HM, BnR, col=['start_lng', 'start_lat'], accuracy=5) #起点在家
# 从6-9点,起点在地铁站的出行中筛选终点是工作地的
MtoW = tbd.clean_outofshape(MtoW, workplace, col=['end_lng', 'end_lat'], accuracy=5) #终点在地铁站

In [126]:
MtoW

,counter,day,dist,end_lat,end_lat_bd,end_lng,end_lng_bd,end_time,hour,minute,start_lat,start_lat_bd,start_lng,start_lng_bd,start_time,time,velocity
0,1155500,2,1188.430000,22.546980,22.550494,114.006841,114.018317,2021/2/2 7:52,7,37,22.536345,22.539855,114.005732,114.017205,2021/2/2 7:37,900,4.753720
1,7161709,7,878.790000,22.536254,22.539789,114.010000,114.025743,2021/2/7 7:38,7,32,22.536345,22.539855,114.005713,114.017186,2021/2/7 7:32,360,8.787900
2,3781944,4,1007.770000,22.525390,22.528703,113.932447,113.943769,2021/2/4 8:38,8,32,22.522807,22.526106,113.924074,113.935390,2021/2/4 8:32,360,10.077700
3,33,1,558.804923,22.534252,22.537583,113.941679,113.953012,2021/2/1 6:06,6,2,22.538488,22.541813,113.938746,113.950075,2021/2/1 6:02,240,8.382074
4,34,1,558.804820,22.534339,22.537670,113.941766,113.953099,2021/2/1 6:06,6,2,22.538575,22.541901,113.938833,113.950163,2021/2/1 6:02,240,8.382072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30005,7188406,7,521.990000,22.550240,22.553890,114.090000,114.100660,2021/2/7 9:42,9,36,22.554288,22.557938,114.091653,114.103233,2021/2/7 9:36,360,5.219900
30006,7188867,7,1227.420000,22.551447,22.555096,114.090000,114.103918,2021/2/7 9:47,9,39,22.562474,22.566124,114.092875,114.104455,2021/2/7 9:39,480,9.205650
30007,7189842,7,1144.340000,22.559276,22.562555,113.900000,113.912712,2021/2/7 9:56,9,48,22.565457,22.568733,113.892474,113.903800,2021/2/7 9:48,480,8.582550
30008,7191475,7,1891.250000,22.549867,22.553498,114.120000,114.127269,2021/2/7 10:11,9,56,22.561198,22.564807,114.129497,114.141025,2021/2/7 9:56,900,7.565000


In [91]:
# 识别起点
# 识别出发地和目的地
# 工作地到地铁站，时间17-20点
start_17_20 = DBS_all[DBS_all['hour'].map(lambda x:17<=x<=20)]

In [127]:
# 从17-20点的出行中筛选起点是在工作地的
WtoM = tbd.clean_outofshape(start_17_20, workplace, col=['start_lng', 'start_lat'], accuracy=5) #起点在家
# 从17-20点,起点在工作地的出行中筛选终点是地铁站的
WtoM = tbd.clean_outofshape(WtoM, BnR, col=['end_lng', 'end_lat'], accuracy=5) #终点在地铁站

In [128]:
WtoM

,start_time,start_lat_bd,end_time,end_lng_bd,start_lng_bd,end_lat_bd,dist,time,velocity,minute,hour,day,start_lng,start_lat,end_lng,end_lat,counter
0,2021/2/1 17:01,22.552510,2021/2/1 17:04,114.092332,114.089531,22.550821,343.556338,180,6.871127,1,17,1,114.077947,22.548861,114.080749,22.547172,359457
1,2021/2/1 17:01,22.552510,2021/2/1 17:04,114.092332,114.089531,22.550821,343.556338,180,6.871127,1,17,1,114.077947,22.548861,114.080749,22.547172,359459
2,2021/2/1 17:01,22.552499,2021/2/1 17:04,114.092321,114.089520,22.550810,343.556357,180,6.871127,1,17,1,114.077937,22.548850,114.080738,22.547161,359460
3,2021/2/1 20:33,22.552489,2021/2/1 20:46,114.103197,114.089504,22.552995,1407.270637,780,6.495095,33,20,1,114.077921,22.548840,114.091617,22.549345,512440
4,2021/2/1 20:35,22.552511,2021/2/1 20:52,114.103760,114.089531,22.557049,1545.849673,1020,5.455940,35,20,1,114.077947,22.548862,114.092180,22.553399,515290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30773,2021/2/7 20:27,22.540517,2021/2/7 20:46,114.068379,114.058583,22.538800,1023.960000,1140,3.233558,27,20,7,114.047024,22.536904,114.060000,22.535171,7266536
30774,2021/2/7 20:37,22.550464,2021/2/7 20:54,114.127930,114.126534,22.544650,662.180000,1020,2.337106,37,20,7,114.114980,22.546833,114.120000,22.541022,7267079
30775,2021/2/7 20:52,22.523669,2021/2/7 20:55,114.051877,114.053677,22.525914,310.700000,180,6.214000,52,20,7,114.042126,22.520066,114.040000,22.522315,7267169
30776,2021/2/7 20:49,22.551400,2021/2/7 20:59,114.102564,114.099011,22.557500,770.200000,600,4.621200,49,20,7,114.087429,22.547750,114.090000,22.553850,7267495


In [129]:
# 剔除WtoM
#上下合并表格，并重置索引
start_17_20_WM = pd.concat([start_17_20,WtoM], ignore_index=True, verify_integrity=True,sort=True) 
#删除合并后姓名里有重复的所有行
start_17_20_WM.drop_duplicates(subset=['counter'],keep=False,inplace=True) #subset=['counter']表示检查counter这一列是否有重复内容，keep=False表示不保留重复内容

In [130]:
start_17_20_WM

,counter,day,dist,end_lat,end_lat_bd,end_lng,end_lng_bd,end_time,hour,minute,start_lat,start_lat_bd,start_lng,start_lng_bd,start_time,time,velocity
0,358160,1,87.710767,22.551600,22.555011,113.972368,113.983759,2021/2/1 17:01,17,0,22.551001,22.554413,113.972923,113.984316,2021/2/1 17:00,60,5.262646
1,358432,1,55.630907,22.619373,22.622910,114.011781,114.023273,2021/2/1 17:01,17,0,22.618989,22.622526,114.012128,114.023621,2021/2/1 17:00,60,3.337854
2,358473,1,297.597316,22.504383,22.507677,113.920698,113.932011,2021/2/1 17:01,17,0,22.504008,22.507298,113.917830,113.929143,2021/2/1 17:00,60,17.855839
3,358497,1,83.681047,22.483128,22.486407,113.905357,113.916669,2021/2/1 17:01,17,0,22.483470,22.486749,113.906083,113.917395,2021/2/1 17:00,60,5.020863
4,358608,1,363.484851,22.537122,22.540771,114.079634,114.091216,2021/2/1 17:02,17,0,22.537211,22.540858,114.076096,114.087678,2021/2/1 17:00,120,10.904546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2098211,7281173,7,804.660000,22.600999,22.604564,114.020000,114.034586,2021/2/7 18:55,18,50,22.598069,22.601651,114.030229,114.041761,2021/2/7 18:50,300,9.655920
2098212,7281174,7,804.660000,22.601072,22.604638,114.020000,114.034660,2021/2/7 18:55,18,50,22.598142,22.601725,114.030303,114.041835,2021/2/7 18:50,300,9.655920
2098213,7281175,7,1932.850000,22.515701,22.519296,114.040000,114.050264,2021/2/7 20:09,19,59,22.501402,22.505018,114.049434,114.060994,2021/2/7 19:59,600,11.597100
2098214,7281176,7,411.390000,22.600904,22.604470,114.020000,114.035004,2021/2/7 18:19,18,17,22.597674,22.601245,114.025443,114.036966,2021/2/7 18:17,120,12.341700


In [131]:
# 从17-20点的出行中筛选起点是在地铁站的
MtoH = tbd.clean_outofshape(start_17_20_WM, BnR, col=['start_lng', 'start_lat'], accuracy=5) #起点在家
# 从17-20点,起点在地铁站的出行中筛选终点是家的
MtoH = tbd.clean_outofshape(MtoH, workplace, col=['end_lng', 'end_lat'], accuracy=5) #终点在地铁站

In [132]:
MtoH

,counter,day,dist,end_lat,end_lat_bd,end_lng,end_lng_bd,end_time,hour,minute,start_lat,start_lat_bd,start_lng,start_lng_bd,start_time,time,velocity
0,2549077,3,403.690000,22.536645,22.540277,114.058830,114.070403,2021/2/3 18:11,18,8,22.539534,22.543162,114.056447,114.068018,2021/2/3 18:08,180,8.073800
1,2971707,3,403.690000,22.536645,22.540277,114.058830,114.070403,2021/2/3 18:11,18,8,22.539534,22.543162,114.056447,114.068018,2021/2/3 18:08,180,8.073800
2,359352,1,248.021672,22.565953,22.569602,114.096908,114.108485,2021/2/1 17:04,17,0,22.564320,22.567968,114.098554,114.110129,2021/2/1 17:00,240,3.720325
3,1502353,2,702.790000,22.565940,22.569589,114.096897,114.108474,2021/2/2 18:13,18,8,22.565911,22.569555,114.103749,114.115319,2021/2/2 18:08,300,8.433480
4,359374,1,227.916040,22.539672,22.543298,114.118174,114.129722,2021/2/1 17:04,17,2,22.540164,22.543793,114.116017,114.127568,2021/2/1 17:02,120,6.837481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28109,7261927,7,919.970000,22.517136,22.520422,113.910000,113.924829,2021/2/7 20:11,20,0,22.508997,22.512284,113.915130,113.926442,2021/2/7 20:00,660,5.018018
28110,7263496,7,851.890000,22.541628,22.545277,114.090000,114.104792,2021/2/7 20:23,20,16,22.549289,22.552938,114.093165,114.104744,2021/2/7 20:16,420,7.301914
28111,7263822,7,564.280000,22.563872,22.567148,113.890000,113.902217,2021/2/7 20:25,20,18,22.565938,22.569215,113.885865,113.897198,2021/2/7 20:18,420,4.836686
28112,7264407,7,877.610000,22.547427,22.551076,114.090000,114.104278,2021/2/7 20:30,20,21,22.554089,22.557739,114.088116,114.099698,2021/2/7 20:21,540,5.850733


In [133]:
HtoM.to_csv('HtoM.csv')
MtoW.to_csv('MtoW.csv')
WtoM.to_csv('WtoM.csv')
MtoH.to_csv('MtoH.csv')